# Importing libraries

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Libraries imported.


# Getting data from wikipedia

In [2]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=946126446"

toronto_data  = requests.get(url).text
toronto_data

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"2ed5bd49-39f5-4562-b646-49551782a8fc","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":945633050,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: discouraged parameter","Articles with short description","Short description is different fro

# Creating a dataframe

In [3]:
soup = BeautifulSoup(toronto_data,"html5lib")  # Parse the data

table_contents=[]

for i in soup.find('table',{"class":"wikitable sortable"}).find('tbody').find_all('tr'):
    row = i.find_all("td")
    cell = {}
    if(row):
        if row[1].text != 'Not assigned': # Not select cells with a borough that is Not assigned.
            cell['PostalCode'] = row[0].text
            cell['Borough'] = row[1].text
            cell['Neighborhood'] = row[2].text
            table_contents.append(cell)
            
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head() 

Borough        Neighborhood PostalCode
0        North York         Parkwoods\n        M3A
1        North York  Victoria Village\n        M4A
2  Downtown Toronto      Harbourfront\n        M5A
3        North York  Lawrence Heights\n        M6A
4        North York    Lawrence Manor\n        M6A

Check cells who have Not assigned neighborhood

In [4]:
nan_neighborhood = (df['Neighborhood'] == 'Not assigned')
df[nan_neighborhood]

Empty DataFrame
Columns: [Borough, Neighborhood, PostalCode]
Index: []

There is no Not assigned neighborhood.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
Borough         210 non-null object
Neighborhood    210 non-null object
PostalCode      210 non-null object
dtypes: object(3)
memory usage: 5.0+ KB


Dataframe has 210 rows.

In [6]:
df['PostalCode'].value_counts()

M9V    8
M8Y    8
M5V    7
M9B    5
M8Z    5
M4V    5
M1V    4
M6M    4
M9R    4
M9C    4
M5H    3
M1M    3
M1E    3
M2J    3
M8X    3
M6L    3
M5J    3
M1P    3
M3H    3
M8V    3
M1C    3
M1K    3
M1L    3
M1T    3
M6K    3
M5T    3
M5R    3
M4L    2
M6P    2
M5L    2
      ..
M7A    1
M4A    1
M4M    1
M5W    1
M6B    1
M2R    1
M6E    1
M1G    1
M4G    1
M3B    1
M4N    1
M3L    1
M4C    1
M5A    1
M2H    1
M5N    1
M3N    1
M1H    1
M2P    1
M4H    1
M4S    1
M4J    1
M4R    1
M7R    1
M4E    1
M3A    1
M1J    1
M1S    1
M1W    1
M6G    1
Name: PostalCode, Length: 103, dtype: int64

More than one neighborhood exist in one postal code area. These rows should be combined into one row with the neighborhoods separated with a comma. 

In [7]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighborhood'] = df['Neighborhood'].str.join(', ')
df['Neighborhood'] = df['Neighborhood'].replace('\n','',regex=True) # Delete \n
df

PostalCode           Borough  \
0          M6P      West Toronto   
1          M3N        North York   
2          M5N   Central Toronto   
3          M5A  Downtown Toronto   
4          M6L        North York   
5          M1N       Scarborough   
6          M1V       Scarborough   
7          M4W  Downtown Toronto   
8          M1M       Scarborough   
9          M1K       Scarborough   
10         M3H        North York   
11         M2R        North York   
12         M2J        North York   
13         M1X       Scarborough   
14         M3A        North York   
15         M9V         Etobicoke   
16         M1J       Scarborough   
17         M5H  Downtown Toronto   
18         M1S       Scarborough   
19         M6J      West Toronto   
20         M1P       Scarborough   
21         M1W       Scarborough   
22         M8W         Etobicoke   
23         M8Y         Etobicoke   
24         M5K  Downtown Toronto   
25         M7A  Downtown Toronto   
26         M1C       Scarborough   
27         M6E              York   
28         M1T       Scarborough   
29         M4N   Central Toronto   
..         ...               ...   
73         M1B       Scarborough   
74         M4B         East York   
75         M5V  Downtown Toronto   
76         M3C        North York   
77         M5M        North York   
78         M6K      West Toronto   
79         M9C         Etobicoke   
80         M6G  Downtown Toronto   
81         M4K      East Toronto   
82         M5X  Downtown Toronto   
83         M4H         East York   
84         M4J         East York   
85         M4T   Central Toronto   
86         M5T  Downtown Toronto   
87         M2M        North York   
88         M4C         East York   
89         M9R         Etobicoke   
90         M4G         East York   
91         M5P   Central Toronto   
92         M4P   Central Toronto   
93         M4R   Central Toronto   
94         M6N              York   
95         M6A        North York   
96         M6C              York   
97         M9M        North York   
98         M4S   Central Toronto   
99         M5E  Downtown Toronto   
100        M8V         Etobicoke   
101        M3L        North York   
102        M5W  Downtown Toronto   

                                          Neighborhood  
0                        High Park, The Junction South  
1                                  Downsview Northwest  
2                                             Roselawn  
3                                         Harbourfront  
4                   Downsview, North Park, Upwood Park  
5                          Birch Cliff, Cliffside West  
6    Agincourt North, L'Amoreaux East, Milliken, St...  
7                                             Rosedale  
8      Cliffcrest, Cliffside, Scarborough Village West  
9          East Birchmount Park, Ionview, Kennedy Park  
10     Bathurst Manor, Downsview North, Wilson Heights  
11                                     Willowdale West  
12                        Fairview, Henry Farm, Oriole  
13                                         Upper Rouge  
14                                           Parkwoods  
15   Albion Gardens, Beaumond Heights, Humbergate, ...  
16                                 Scarborough Village  
17                            Adelaide, King, Richmond  
18                                           Agincourt  
19                            Little Portugal, Trinity  
20   Dorset Park, Scarborough Town Centre, Wexford ...  
21                                     L'Amoreaux West  
22                              Alderwood, Long Branch  
23   Humber Bay, King's Mill Park, Kingsway Park So...  
24            Design Exchange, Toronto Dominion Centre  
25                                        Queen's Park  
26              Highland Creek, Rouge Hill, Port Union  
27                                 Caledonia-Fairbanks  
28             Clarks Corners, Sullivan, Tam O'Shanter  
29                                       Lawrence Park  
..  

In [8]:
df.shape

(103, 3)